Hello guys we are a team of 3 girls learning to do machine learning. We have put this notebook public for insight on how to improve our models. Please know that the models we used are taken from other kaggle models in this thread. Thus, some parts are strongly inspired from these other models. We are now trying to twerk the code to make it our own and learn how we can improve these models. Big thanks to :

[Data preprocessing && resampling][1]

[Over+Under+SMOTE][2]

[Model train + why not ROC][3]

  [1]: https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now
  [2]: https://www.kaggle.com/gargmanish/how-to-handle-imbalance-data-study-in-detail
  [3]: https://www.kaggle.com/lct14558/imbalanced-data-why-you-should-not-use-roc-curve

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC # SVM
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report

%matplotlib inline

In [2]:
data = pd.read_csv("/Users/zahrakhambaty/Downloads/creditcard.csv",header = 0)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
## Pre-processing the data
## Normalizing the amount column

from sklearn.preprocessing import StandardScaler
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))
data = data.drop(['Time','Amount'],axis=1)
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [4]:
## Since the data is largely imbalanced we need to resample the data such that the proportion/ratio between fraudulent and normal transactions are relativeley similar.

x = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']

In [5]:
#UNDERSAMPLING
# Number of fraudelent transaction in the existing data
numberOffraudulentTransaction = len(data[data.Class == 1])
fraudIndices = np.array(data[data.Class == 1].index)

# Picking the indices of the normal classes
normalIndices = data[data.Class == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normalIndices, numberOffraudulentTransaction, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraudIndices,random_normal_indices])

# Under sample dataset
under_sample_data = data.iloc[under_sample_indices,:]

x_undersample = under_sample_data.loc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.loc[:, under_sample_data.columns == 'Class']

# Showing ratio
print("Percentage of normal transactions: ", len(under_sample_data[under_sample_data.Class == 0])/len(under_sample_data))
print("Percentage of fraud transactions: ", len(under_sample_data[under_sample_data.Class == 1])/len(under_sample_data))
print("Total number of transactions in resampled data: ", len(under_sample_data))


Percentage of normal transactions:  0.5
Percentage of fraud transactions:  0.5
Total number of transactions in resampled data:  984


In [6]:

## Splitting the data into Training,Validation and Test Set##
## Test Set needs to be unused till the mere end##
X_train, X_test, Y_train, Y_test = train_test_split(data,y, test_size=0.25, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.25, random_state=42)
# #Figuring out the ratio of normal transction and fraudelent transaction from training data# #
normal_tdata = X_train[X_train["Class"]==0]
print("train data: length of normal data",len(normal_tdata))
fraud_tdata = X_train[X_train["Class"]==1]
print("train data: length of fraud data",len(fraud_tdata))
## dataset for validation set ##
normal_vdata = X_val[X_val["Class"]==0]
print("For Validation Set :length of normal data",len(normal_vdata))
fraud_vdata = X_val[X_val["Class"]==1]
print("For Validation Set :length of fraud data",len(fraud_vdata))

train data: length of normal data 159900
train data: length of fraud data 303
For Validation Set :length of normal data 53320
For Validation Set :length of fraud data 82


In [7]:
#SMOTE #OVERSAMPLING
#Since the data is highly imbalanced we use the imblearn to balance out the data by introducing more fraudulent data ##
#basically oversampling of data but with synthesized data so no duplicates
sm = SMOTE(random_state=12, ratio = 0.6, k_neighbors=5)
x_train_res, y_train_res = sm.fit_sample(X_train, Y_train.values.ravel())

In [8]:
a = x_train_res[:,28]
b= np.count_nonzero(a == 1)
c= np.count_nonzero(a == 0)
print("length of oversampled data is ",len(x_train_res))
print("Number of normal transcation in oversampled data",b)
print("No.of fraud transcation",c)
print("Proportion of Normal data in oversampled data is ",c/len(x_train_res))
print("Proportion of fraud data in oversampled data is ",b/len(x_train_res))

length of oversampled data is  255840
Number of normal transcation in oversampled data 95940
No.of fraud transcation 159900
Proportion of Normal data in oversampled data is  0.625
Proportion of fraud data in oversampled data is  0.375


In [9]:
print ("UNDERSAMPLING")
df = under_sample_data
#train, validate, test = np.split(df.sample(frac=1), [int(.5*len(df)), int(.75*len(df))])
x, x_test, y, y_test = train_test_split(x_undersample,y_undersample,test_size=0.25,train_size=0.75)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.33,train_size =0.66)

scaler = StandardScaler()

#Get mean+average and standardize to Z
x_train = scaler.fit_transform (x_train)

#Apply same transformation to hidden data
x_cv = scaler.transform (x_cv)
x_test = scaler.transform (x_test)

# cross-validate needs to be here (after the splitting for proper X-V)

UNDERSAMPLING


In [10]:
#UNDERSAMPLING
# My logic is regressing, guys!
logi = LogisticRegression(class_weight='balanced')
mdl = logi.fit(x_train, y_train.values.ravel())
predictions = logi.predict(x_test)
print(accuracy_score(y_test, predictions))
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))
print("LOGISTIC REGRESSION CLASS WEIGHT BALANCED")


# Support vector machine, boss!
print("SVM")

#Other models doing 75%
svc = SVC(C=1, kernel='linear')
svc.fit (x_train,y_train.values.ravel())
ypredsvc = svc.predict (x_test)
print(confusion_matrix(y_test, ypredsvc))
print(f1_score(y_test, ypredsvc))


# Random Forest stories, mate!

classif = RandomForestClassifier(n_estimators=100, n_jobs=2, min_samples_split=2, random_state=0)
#estimator = nb of free in forest, nbjobs = parallel calcul using cpu
#scores = cross_val_score(clf, X, y)
#scores.mean()    
classif.fit(x_train, y_train.values.ravel())
y_pred_test_rf = classif.predict(x_test)
print("RANDOMFOREST")
print(confusion_matrix(y_test, y_pred_test_rf))
print(f1_score(y_test, y_pred_test_rf))


classif = RandomForestClassifier(n_estimators=100, n_jobs=2, min_samples_split=2, random_state=42)
#estimator = nb of free in forest, nbjobs = parallel calcul using cpu
#scores = cross_val_score(clf, X, y)
#scores.mean()    
classif.fit(x_train, y_train.values.ravel())
y_pred_test_rf = classif.predict(x_test)
print("RANDOMFOREST3")
print(confusion_matrix(y_test, y_pred_test_rf))
print(f1_score(y_test, y_pred_test_rf))

classif = RandomForestClassifier(n_estimators=100, n_jobs=5, min_samples_split=2, random_state=0)
#estimator = nb of free in forest, nbjobs = parallel calcul using cpu
#scores = cross_val_score(clf, X, y)
#scores.mean()    
classif.fit(x_train, y_train.values.ravel())
y_pred_test_rf = classif.predict(x_test)
print("RANDOMFOREST4")
print(confusion_matrix(y_test, y_pred_test_rf))
print(f1_score(y_test, y_pred_test_rf))


# Decision Tree, baby!
classif2 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
classif2.fit(x_train, y_train.values.ravel())
y_pred_test_clf2 = classif2.predict(x_test)
#scores = cross_val_score(clf, x_train, y_train)
#scores.mean()
print("DECISIONTREE1")
print(confusion_matrix(y_test, y_pred_test_clf2))
print(f1_score(y_test, y_pred_test_clf2))

classif2 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=15)
classif2.fit(x_train, y_train.values.ravel())
y_pred_test_clf2 = classif2.predict(x_test)
#scores = cross_val_score(clf, x_train, y_train)
#scores.mean()
print("DECISIONTREE2")
print(confusion_matrix(y_test, y_pred_test_clf2))
print(f1_score(y_test, y_pred_test_clf2))


# LOGISTIC REGRESSION,TUNING THE PARAMETERS TO SEE THE RESULTS, captain!
lr = LogisticRegression(C = 1, penalty = 'l1')
lr.fit(x_train, y_train.values.ravel())
y_pred_test_nn = lr.predict(x_test)
print("LOGISTIC REGRESSION PAREMETER 1")
print(accuracy_score(y_test, predictions))
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))

lr = LogisticRegression(C = 50, penalty = 'l1')
lr.fit(x_train, y_train.values.ravel())
y_pred_test_nn = lr.predict(x_test)
print("LOGISTIC REGRESSION PAREMETER 50")
print(accuracy_score(y_test, y_pred_test_nn))
print (confusion_matrix(y_test, y_pred_test_nn))
print (classification_report(y_test, y_pred_test_nn))

#l1 and l2 gives very similar result
lr = LogisticRegression(C = 500, penalty = 'l1')
lr.fit(x_train, y_train.values.ravel())
y_pred_test_nn = lr.predict(x_test)
print("LOGISTIC REGRESSION PAREMETER 500")
print(accuracy_score(y_test, y_pred_test_nn))
print (confusion_matrix(y_test, y_pred_test_nn))
print (classification_report(y_test, y_pred_test_nn))




0.959349593496
[[127   3]
 [  7 109]]
             precision    recall  f1-score   support

          0       0.95      0.98      0.96       130
          1       0.97      0.94      0.96       116

avg / total       0.96      0.96      0.96       246

LOGISTIC REGRESSION CLASS WEIGHT BALANCED
SVM
[[127   3]
 [ 10 106]]
0.942222222222
RANDOMFOREST
[[126   4]
 [  9 107]]
0.942731277533
RANDOMFOREST3
[[126   4]
 [  9 107]]
0.942731277533
RANDOMFOREST4
[[126   4]
 [  9 107]]
0.942731277533
DECISIONTREE1
[[119  11]
 [ 10 106]]
0.909871244635
DECISIONTREE2
[[121   9]
 [ 10 106]]
0.917748917749
LOGISTIC REGRESSION PAREMETER 1
0.959349593496
[[127   3]
 [  7 109]]
             precision    recall  f1-score   support

          0       0.95      0.98      0.96       130
          1       0.97      0.94      0.96       116

avg / total       0.96      0.96      0.96       246

LOGISTIC REGRESSION PAREMETER 50
0.959349593496
[[125   5]
 [  5 111]]
             precision    recall  f1-score   sup

In [11]:

print ("OVERSAMPLING")

#substract the min of the column divide by the max  for the whole column
#also apply on test+validation
#try without normalizing at all, nor stabilize

# LOGICREGRESSION, captain!

logi = LogisticRegression(class_weight='balanced')
mdl = logi.fit(x_train_res, y_train_res)
predictions2 = logi.predict(X_test)
print(accuracy_score(Y_test, predictions2))
print (confusion_matrix(Y_test, predictions2))
print (classification_report(Y_test, predictions2))
print("LOGICREGRESSION WITH Balanced weight ")

# LOGICREGRESSION, captain!
lr = LogisticRegression(C = 5, penalty = 'l1')
lr.fit(x_train_res,y_train_res)
Y_pred_test_nn = lr.predict(X_test)
print("LOGICREGRESSION WITH C=5 ")
print(confusion_matrix(Y_test, Y_pred_test_nn))
print(f1_score(Y_test, Y_pred_test_nn))


# Random Forest stories, mate!
classif = RandomForestClassifier(n_estimators=100, n_jobs=2, min_samples_split=2, random_state=0)
#estimator = nb of free in forest, nbjobs = parallel calcul using cpu
#scores = cross_val_score(clf, X, y)
#scores.mean()    
classif.fit(x_train_res, y_train_res)
Y_pred_test_rf = classif.predict(X_test)
print("RANDOMFOREST")
print(confusion_matrix(Y_test, Y_pred_test_rf))
print(f1_score(Y_test, Y_pred_test_rf))

# Decision Tree, baby!
classif2 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
classif2.fit(x_train_res, y_train_res)
Y_pred_test_clf2 = classif2.predict(X_test)
#scores = cross_val_score(clf, x_train, y_train)
#scores.mean()
print("DECISIONTREE")
print(confusion_matrix(Y_test, Y_pred_test_clf2))
print(f1_score(Y_test, Y_pred_test_clf2))

## Neural Network
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x_train_res, y_train_res)
y_pred_test_neural =clf.predict_proba(X_test) 
print("Neural Network")
print( y_pred_test_neural)
#print(f1_score(Y_test, y_pred_test_neural))



OVERSAMPLING
1.0
[[71095     0]
 [    0   107]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     71095
          1       1.00      1.00      1.00       107

avg / total       1.00      1.00      1.00     71202

LOGICREGRESSION WITH Balanced weight 
LOGICREGRESSION WITH C=5 
[[71095     0]
 [    0   107]]
1.0
RANDOMFOREST
[[71095     0]
 [    0   107]]
1.0
DECISIONTREE
[[71095     0]
 [    0   107]]
1.0
Neural Network
[[  9.99991461e-01   8.53883196e-06]
 [  9.99991461e-01   8.53883196e-06]
 [  9.99991461e-01   8.53883196e-06]
 ..., 
 [  9.99991461e-01   8.53883196e-06]
 [  9.99991461e-01   8.53883196e-06]
 [  9.99991461e-01   8.53883196e-06]]


In [12]:

## Logistic/undersampling
print ("Logistic with Undersampling")
tn, fp, fn,tp = confusion_matrix(predictions,y_test).ravel() 

Sensitivity=tp/float((tp+fn))#Sensitivity 
print ("SENS ",Sensitivity)

Specificity=tn/float((tn+fp))#Specificity 
print ("SPEC",Specificity)

Accuracy= accuracy_score(predictions,y_test, normalize=True, sample_weight=None)
print ("ACC",Accuracy)

## Logistic/Oversampling

print ("Logistic with Oversampling")
tn, fp, fn,tp = confusion_matrix(predictions2,Y_test).ravel() 
Sensitivity=tp/float((tp+fn))#Sensitivity 
print ("SENS",Sensitivity)

Specificity=tn/float((tn+fp))#Specificity 
print ("SPEC",Specificity)

Accuracy= accuracy_score(predictions2,Y_test, normalize=True, sample_weight=None)
print ("ACC",Accuracy)

Logistic with Undersampling
SENS  0.973214285714
SPEC 0.94776119403
ACC 0.959349593496
Logistic with Oversampling
SENS 1.0
SPEC 1.0
ACC 1.0
